### read .dcm and save as .npy

In [ ]:
from glob import glob

import numpy as np
import dicom
import os

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

In [ ]:
def dcm2npy(in_path, out_path, out_file_name = "fullimages.npy"
            max_setter = -1024, set_max_setter_value = -1024, 
            upper_range = -600, lower_range = 500, set_range_value = 1000):
    '''
    in_path : import dcm 2d stack data path
    out_path : export npy data path
    out_file_name : set exported file name
    max_setter보다 낮은 pixel 값은 모두 set_max_setter_value로 바꿈 ... air 및 CT circle을 지우기 위함
    upper_range보다 크고 lower_range보다 작은 사이의 pixel 값을 set_range_value로 바꿈 ... 특정 값을 높이기 위함
    '''
    data_path = in_path
    output_path = out_path
    g = glob(data_path + '/*.dcm')
    
    # Print out the first 5 file names to verify we're in the right folder.
    print("Total of %d DICOM images.\nFirst 5 filenames:" % len(g))
    print('\n'.join(g[:5]))

    patient = load_scan(data_path)
    imgs = get_pixels_hu(patient
                         max_setter, set_max_setter_value,
                         upper_range, lower_range, set_range_value)
    
    output_full_path = output_path + out_file_name
    
    np.save(output_full_path, imgs)
    
    imgs_data_dcm_to_npy = np.load(output_full_path)
    
    return imgs_data_dcm_to_npy

def load_scan(path):
    slices = [dicom.read_file(path + '/' + s) for s in os.listdir(path)]
    slices.sort(key = lambda x: int(x.InstanceNumber))
    try:
        slice_thickness = np.abs(slices[0].ImagePositionPatient[2] - slices[1].ImagePositionPatient[2])
    except:
        slice_thickness = np.abs(slices[0].SliceLocation - slices[1].SliceLocation)
        
    for s in slices:
        s.SliceThickness = slice_thickness
        
    return slices

def get_pixels_hu(scans, 
                  max_setter, set_max_setter_value,
                  upper_range, lower_range, set_range_value):
    # hounsfield unit
    image = np.stack([s.pixel_array for s in scans])
    
    # Convert to int16 (from sometimes int16), 
    # should be possible as values should always be low enough (<32k)
    image = image.astype(np.int16)

    # Set outside-of-scan pixels to 1
    # The intercept is usually -1024, so air is approximately 0
    # saturated air in CT
    image[image <= max_setter] = set_max_setter_value
    image[(image > upper_range) & (image < lower_range)] = set_range_value
    
    # Convert to Hounsfield units (HU)
    intercept = scans[0].RescaleIntercept
    slope = scans[0].RescaleSlope
    
    if slope != 1:
        image = slope * image.astype(np.float64)
        image = image.astype(np.int16)
        
    image += np.int16(intercept)
    
    return np.array(image, dtype=np.int16)

### load .npy data and check 2D stack dicom image

In [ ]:
def sample_stack(stack, rows=6, cols=6, start_with=10, show_every=5):
    fig,ax = plt.subplots(rows,cols,figsize=[12,12])
    for i in range(rows*cols):
        ind = start_with + i*show_every
        ax[int(i/rows),int(i % rows)].set_title('slice %d' % ind)
        ax[int(i/rows),int(i % rows)].imshow(stack[ind],cmap='gray')
        ax[int(i/rows),int(i % rows)].axis('off')
    plt.show()

In [ ]:
dcm_data_path = 'C:/Users/dDk13/Desktop/ddk_test/PointCloudTest/data/manekin_data/'
out_data_path = '01_export_data/'

imgs_dicom_to_process = dcm2npy(dcm_data_path, out_data_path)
num_of_dicom_data = len(imgs_dicom_to_process)

In [ ]:
# check a image
sample_stack(imgs_dicom_to_process)

### eliminate half of image (lower)

 - 환자의 표면부만 검출하고할 때만 사용!

In [ ]:
def blocked_garbage(imgs_to_process):
    
    upper = np.ndarray(shape=(256,512), dtype=np.int16)
    lower = np.ndarray(shape=(256,512), dtype=np.int16)

    upper.fill(1)
    lower.fill(0)

    block_filter = np.concatenate((upper, lower), axis=0)

    img_data_blocked = []

    for i in range(len(imgs_to_process)):
        img_tmp = imgs_dicom_to_process[i]
        img_tmp_blocked = img_tmp * block_filter
    
        img_tmp_blocked[img_tmp_blocked == 0] = -1024
    
        img_data_blocked.append(img_tmp_blocked)
        
    return np.array(img_data_blocked)

In [ ]:
dcm_data_blocked = blocked_garbage(imgs_dicom_to_process)

In [ ]:
# check a image
sample_stack(dcm_data_blocked)

### contour detection (dilation - find contour - erosion)

In [ ]:
from skimage.morphology import disk, dilation, erosion
from skimage import measure
from PIL import Image, ImageDraw

def contour_detection(imgs_to_process):
    num_of_img_data = len(imgs_to_process)
    imgs_to_process_dilation = []

    for i in range(num_of_img_data):
        mask = disk(3)
        img = dilation(imgs_to_process[i], selem=mask)
        imgs_to_process_dilation.append(img)
        
    imgs_to_process_contour = []

    for i in range(num_of_img_data):

        # Construct some test data
        r = imgs_to_process_dilation[i]

        # Find contours at a constant value of 0.8
        contours = measure.find_contours(r, 0.8)

        contour_surf = contours[0]

        polygon = []

        for p in range(len(contour_surf)):
            polygon.append((contour_surf[p][1], contour_surf[p][0]))

        # polygon = [(x1,y1),(x2,y2),...]
        width = 512
        height = 512

        img = Image.new('L', (width, height), 0)
        ImageDraw.Draw(img).polygon(polygon, outline=1, fill=1)
        mask = np.array(img)

        mask_ers = erosion(mask, selem=disk(3))

        imgs_to_process_contour.append(mask_ers)
    
    return np.array(imgs_to_process_contour)

In [ ]:
dcm_data_contour = contour_detection(dcm_data_blocked)

In [ ]:
# check a image
sample_stack(dcm_data_contour)

### make 3D data (.obj and .ply)

In [ ]:
from skimage import morphology
from skimage.transform import resize
from sklearn.cluster import KMeans
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.tools import FigureFactory as FF
from plotly.graph_objs import *

def make_mesh(image, threshold=-300, step_size=1):

    print("Transposing surface")
    p = image.transpose(2,1,0)
    
    print("Calculating surface")
    verts, faces, norm, val = measure.marching_cubes_lewiner(p, threshold, step_size=step_size, allow_degenerate=True) 
    return verts, faces

def make_and_save_mesh_to_obj(image, threshold=-300, step_size=1, slice_thickness=1.0):

    print("Transposing surface")
    p = image.transpose(2,1,0)
    
    print("Calculating surface")
    verts, faces, normals, val = measure.marching_cubes_lewiner(p, threshold, spacing=(1.0, 1.0, slice_thickness), 
                                                                step_size=step_size, allow_degenerate=True) 
    faces = faces + 1
    
    thefile = open('test__.obj', 'w')
    for item in verts:
        thefile.write("v {0} {1} {2}\n".format(item[0],item[1],item[2]))

    for item in normals:
        thefile.write("vn {0} {1} {2}\n".format(item[0],item[1],item[2]))

    for item in faces:
        thefile.write("f {0}//{0} {1}//{1} {2}//{2}\n".format(item[0],item[1],item[2]))  

    thefile.close()
    
    print("make .obj fin.")
    
    return True

def make_and_save_mesh_to_ply(image, threshold=-300, step_size=1, slice_thickness=1.0):

    print("Transposing surface")
    p = image.transpose(2,1,0)
    
    print("Calculating surface")
    verts, faces, normals, val = measure.marching_cubes_lewiner(p, threshold, spacing=(1.0, 1.0, slice_thickness), 
                                                                step_size=step_size, allow_degenerate=True) 
    
    hd_0 = "ply\n"
    hd_1 = "format ascii 1.0\n"
    hd_2 = "element vertex {0}\n".format(str(len(verts)))
    hd_3 = "property float x\n"
    hd_4 = "property float y\n"
    hd_5 = "property float z\n"
    hd_6 = "end_header\n"
    
    thefile = open('test__.ply', 'w')
    
    thefile.write(hd_0)
    thefile.write(hd_1)
    thefile.write(hd_2)
    thefile.write(hd_3)
    thefile.write(hd_4)
    thefile.write(hd_5)
    thefile.write(hd_6)
    
    for item in verts:
        thefile.write("{0} {1} {2}\n".format(item[0],item[1],item[2]))  

    thefile.close()
    
    print("make .ply fin.")
    
    return True

def plotly_3d(verts, faces):
    x,y,z = zip(*verts) 
    
    print("Drawing")
    
    # Make the colormap single color since the axes are positional not intensity. 
#    colormap=['rgb(255,105,180)','rgb(255,255,51)','rgb(0,191,255)']
    colormap=['rgb(236, 236, 212)','rgb(236, 236, 212)']
    
    fig = FF.create_trisurf(x=x,
                        y=y, 
                        z=z, 
                        plot_edges=False,
                        colormap=colormap,
                        simplices=faces,
                        backgroundcolor='rgb(64, 64, 64)',
                        title="Interactive Visualization")
    iplot(fig)

def plt_3d(verts, faces):
    print("Drawing")
    x,y,z = zip(*verts) 
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection='3d')

    # Fancy indexing: `verts[faces]` to generate a collection of triangles
    mesh = Poly3DCollection(verts[faces], linewidths=0.05, alpha=1)
    face_color = [1, 1, 0.9]
    mesh.set_facecolor(face_color)
    ax.add_collection3d(mesh)

    ax.set_xlim(0, max(x))
    ax.set_ylim(0, max(y))
    ax.set_zlim(0, max(z))
    ax.set_facecolor((0.7, 0.7, 0.7))
    plt.show()